In [66]:
import pandas as pd

In [67]:
# Função para baixar e tratar tabelas do FBref
def get_fbref_table(url):
    tables = pd.read_html(url, header=[0, 1])
    df = tables[0]
    # Junta MultiIndex em nomes simples
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    # Remove espaços extras
    df.columns = df.columns.str.strip()
    return df

In [ ]:
#ESTATISTICA DE CHUTES

In [74]:
# 2. Estatísticas de chutes
url_shots = "https://fbref.com/en/comps/9/2024-2025/shooting/2024-2025-Premier-League-Stats"
df_shots = get_fbref_table(url_shots)

In [75]:
# Conferir colunas para saber o nome real
print(df_shots.columns.tolist())

['Unnamed: 0_level_0 Squad', 'Unnamed: 1_level_0 # Pl', 'Unnamed: 2_level_0 90s', 'Standard Gls', 'Standard Sh', 'Standard SoT', 'Standard SoT%', 'Standard Sh/90', 'Standard SoT/90', 'Standard G/Sh', 'Standard G/SoT', 'Standard Dist', 'Standard FK', 'Standard PK', 'Standard PKatt', 'Expected xG', 'Expected npxG', 'Expected npxG/Sh', 'Expected G-xG', 'Expected np:G-xG']


In [76]:
#Renomear colunas
df_shots = df_shots.rename(columns={
    'Unnamed: 0_level_0 Squad': 'Time',
    'Standard SoT': 'Chutes no Alvo'
})

df_shots = df_shots[['Time', 'Chutes no Alvo']]


In [77]:
# Conferir colunas para saber o nome real
print(df_shots.columns.tolist())

['Time', 'Chutes no Alvo']


In [78]:
# Seleciona apenas colunas de interesse
df_shots = df_shots[['Time', 'Chutes no Alvo']]

# VISUALIZAR GOLS MARCADOS E SOBRIDOS EM CASA E FORA

In [79]:
# URL da tabela de classificação
url_class_gols = "https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats"

In [80]:
# Lê todas as tabelas da página
tabelas_class_gols = pd.read_html(url_class_gols)

In [81]:
# Tabela correta: índice 0
df_class_gols = tabelas_class_gols[1]

In [117]:
# Conferir colunas para saber o nome real
print(df_class_gols.columns.tolist())

[('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Squad'), ('Home', 'MP'), ('Home', 'W'), ('Home', 'D'), ('Home', 'L'), ('Home', 'GF'), ('Home', 'GA'), ('Home', 'GD'), ('Home', 'Pts'), ('Home', 'Pts/MP'), ('Home', 'xG'), ('Home', 'xGA'), ('Home', 'xGD'), ('Home', 'xGD/90'), ('Away', 'MP'), ('Away', 'W'), ('Away', 'D'), ('Away', 'L'), ('Away', 'GF'), ('Away', 'GA'), ('Away', 'GD'), ('Away', 'Pts'), ('Away', 'Pts/MP'), ('Away', 'xG'), ('Away', 'xGA'), ('Away', 'xGD'), ('Away', 'xGD/90')]


In [151]:
# Seleciona apenas as colunas de interesse
df_gols_casa_fora = df_class_gols[ [('Unnamed: 1_level_0', 'Squad'),('Home', 'GF'), ('Home', 'GA'),('Away', 'GF'),('Away', 'GA')] ]

In [119]:
# Renomear as colunas com dois níveis
df_gols_casa_fora.columns = pd.MultiIndex.from_tuples([
    ("", "Time"),
    ("CASA", "Gols Marcados"),
    ("CASA", "Gols Sofridos"),
    ("FORA", "Gols Marcados"),
    ("FORA", "Gols Sofridos")
])


In [138]:
# Achatar as colunas MultiIndex para facilitar o merge
df_gols_casa_fora_flat = df_gols_casa_fora.copy()
df_gols_casa_fora_flat.columns = [
    "".join(col).strip() if isinstance(col, tuple) else col
    for col in df_gols_casa_fora_flat.columns
]


In [152]:
print(df_gols_casa_fora_flat.columns.tolist())



['Time', 'CASAGols Marcados', 'CASAGols Sofridos', 'FORAGols Marcados', 'FORAGols Sofridos']


In [156]:
df_gols_casa_fora_flat ['TOTALGols Marcados'] = df_gols_casa_fora_flat['CASAGols Marcados'] + df_gols_casa_fora_flat['FORAGols Marcados']


In [158]:
df_gols_casa_fora_flat ['TOTALGols Sofridos'] = df_gols_casa_fora_flat['CASAGols Sofridos'] + df_gols_casa_fora_flat['FORAGols Sofridos']

In [ ]:
# Exclui a coluna 'Total_Gols' do df_final
#df_gols_casa_fora_flat = df_gols_casa_fora_flat.drop(columns=['Total_Gols'])

In [163]:
# Primeiro merge entre df_gols_casa_fora e df_gols
df_final = pd.merge(df_gols_casa_fora_flat, df_shots, on='Time', how='inner')


In [165]:
(df_final.head())

,Time,CASAGols Marcados,CASAGols Sofridos,FORAGols Marcados,FORAGols Sofridos,TOTALGols Marcados,TOTALGols Sofridos,Chutes no Alvo
0,Liverpool,42,16,44,25,86,41,215
1,Arsenal,35,17,34,17,69,34,178
2,Manchester City,43,23,29,21,72,44,208
3,Chelsea,35,18,29,25,64,43,203
4,Newcastle Utd,40,20,28,27,68,47,163


In [173]:
# Ajusta os nomes conforme as tabelas
new_cols = [
    ("", "Time"),
    ("CASA", "Gols Marcados"),
    ("CASA", "Gols Sofridos"),
    ("FORA", "Gols Marcados"),
    ("FORA", "Gols Sofridos"),
    ("TOTAL", "Gols Marcados"),
    ("TOTAL", "Gols Sofridos"), 
    ("", "Chutes no Alvo")
]

In [174]:
# Aplica os novos nomes ao DataFrame final
df_final.columns = pd.MultiIndex.from_tuples(new_cols)

In [175]:
(df_final.head())


CASA                        FORA                \
              Time Gols Marcados Gols Sofridos Gols Marcados Gols Sofridos   
0        Liverpool            42            16            44            25   
1          Arsenal            35            17            34            17   
2  Manchester City            43            23            29            21   
3          Chelsea            35            18            29            25   
4    Newcastle Utd            40            20            28            27   

          TOTAL                               
  Gols Marcados Gols Sofridos Chutes no Alvo  
0            86            41            215  
1            69            34            178  
2            72            44            208  
3            64            43            203  
4            68            47            163

In [185]:
#VISUALIZAR NO NAVEGADOR WEB
# 1. Adiciona estilo na tabela
styled_html = (
    df_final.style
    .hide(axis="index")  # oculta o índice
    .set_table_attributes('border="1" class="dataframe"')
    .set_caption("Premier League 2024/2025")
    .set_table_styles([
        {
            'selector': 'caption',
            'props': [
                ('caption-side', 'top'),   # posição da legenda
                ('font-size', '20px'),     # tamanho da fonte
                ('font-weight', 'bold'),   # opcional: negrito
                ('text-align', 'center')   # centralizado
            ]
        }
    ])
    .set_properties(**{
        'border': '1px solid black',
        'text-align': 'center',
        'font-family': 'Arial',
        'font-size': '14px'
    })
    .to_html()
)

# 2. Salva em HTML com UTF-8-SIG para acentos corretos
with open('index.html', 'w', encoding='utf-8-sig') as f:
    f.write(styled_html)

print("Arquivo HTML salvo com sucesso! Abra no navegador para visualizar.")

Arquivo HTML salvo com sucesso! Abra no navegador para visualizar.


In [69]:


url_classificacao = "https://fbref.com/en/comps/9/Premier-League-Stats"
tabelas_class = pd.read_html(url_classificacao)

# Mostra quantas tabelas existem e os nomes das colunas
for i, tabela in enumerate(tabelas_class):
    print(f"\nTabela {i}:\n", tabela.columns.tolist())



Tabela 0:
 ['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP', 'xG', 'xGA', 'xGD', 'xGD/90', 'Attendance', 'Top Team Scorer', 'Goalkeeper', 'Notes']

Tabela 1:
 [('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Squad'), ('Home', 'MP'), ('Home', 'W'), ('Home', 'D'), ('Home', 'L'), ('Home', 'GF'), ('Home', 'GA'), ('Home', 'GD'), ('Home', 'Pts'), ('Home', 'Pts/MP'), ('Home', 'xG'), ('Home', 'xGA'), ('Home', 'xGD'), ('Home', 'xGD/90'), ('Away', 'MP'), ('Away', 'W'), ('Away', 'D'), ('Away', 'L'), ('Away', 'GF'), ('Away', 'GA'), ('Away', 'GD'), ('Away', 'Pts'), ('Away', 'Pts/MP'), ('Away', 'xG'), ('Away', 'xGA'), ('Away', 'xGD'), ('Away', 'xGD/90')]
